#### **Importing the Neccassary Libraries**

In [1]:
import numpy as np 
import pandas as pd
import requests
from pandas.io.json import json_normalize
import folium 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### **Setting Up credentials for the 4Square API**

In [3]:
CLIENT_ID = 'PRFN0JQ0NG00SOQ522DCVJOUURADAZQ3CH2C1DA5K4ERRJMZ' 
CLIENT_SECRET = 'WYMELWNWQLVOUZXSLXOSTHEUEKTXNCU1FIEWF23T22PNN01H'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: PRFN0JQ0NG00SOQ522DCVJOUURADAZQ3CH2C1DA5K4ERRJMZ


#### **Fetching Data's using the API**

I have passed the city names in venue with near parameter for mobile shop category in that particular city

In [4]:
LIMIT = 500
cities = ['Chennai', 'Hyderabad', 'Bangalore', 'Mumbai', 'Delhi']
Mobile_shop_category = '4f04afc02fb6e1c99f3db0bc'
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        Mobile_shop_category)
    results[city] = requests.get(url).json()

#### **Preprocessing and Formatting the data**

In [7]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']
    
#concating all the 5 city data's into csv    
df1 = pd.DataFrame.from_dict(df_venues['Chennai'])
df1['City'] = 'Chennai'
df2 = pd.DataFrame.from_dict(df_venues['Hyderabad'])
df2['City'] = 'Hyderabad'
df3 = pd.DataFrame.from_dict(df_venues['Bangalore'])
df3['City'] = 'Bangalore'
df4 = pd.DataFrame.from_dict(df_venues['Mumbai'])
df4['City'] = 'Mumbai'
df5 = pd.DataFrame.from_dict(df_venues['Delhi'])
df5['City'] = 'Delhi'
df_cities = pd.concat([df1,df2,df3,df4,df5])
df_cities.reset_index(inplace=True, drop=True)
df_cities.to_csv('Mobile shop data.csv', index=False)

In [41]:
df_cities.head()

,Name,Address,Lat,Lng,City
0,Ritche Street,Ritchie St,13.069477,80.271374,Chennai
1,Croma,"2nd Ave,",13.084631,80.216891,Chennai
2,Reliance Digital Express,Anna Nagar,13.084756,80.213343,Chennai
3,Vodafone Store,Anna nagar,13.085272,80.203079,Chennai
4,Aircel Store,"Aircel Towers, No.301,Poonamalle High Road,Kil...",13.075956,80.236309,Chennai


#### **Data Visualization of mobile shops in each city using folium maps**

In [20]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#dd0000',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  

In [23]:
print("Mobile shops in Chennai")
maps[cities[0]]

Mobile shops in Chennai


In [25]:
print("Mobile shops in Hyderabad")
maps[cities[1]]

Mobile shops in Hyderabad


In [28]:
print('Mobile shops in Bengaluru')
maps[cities[2]]

Mobile shops in Bengaluru


In [30]:
print('Mobile shops in Mumbai')
maps[cities[3]]

Mobile shops in Mumbai


In [31]:
print('Mobile shops in Delhi')
maps[cities[4]]

Mobile shops in Delhi


#### Obviously, Delhi has the highest number of mobile shops.

Now, we need to find the central hub point in delhi to start our EXPO-2020

#### **Finding the Central Point and Measuring the distance for each cities**

In [161]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()]
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#dd0000',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    label = folium.Popup("Central Point", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#dd0000',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])
    print(city)
    print("Distance from the centre point : ",end='')
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Chennai
Distance from the centre point : 0.05105306360595714
Hyderabad
Distance from the centre point : 0.043075877532875756
Bangalore
Distance from the centre point : 0.035131045589305084
Mumbai
Distance from the centre point : 0.08167181371420372
Delhi
Distance from the centre point : 0.09574720569597839


#### *It seems that distance from centre point to other mobile shops are far away in Delhi*

#### *Let's visualize delhi in graph*

In [162]:
print('Distance from Central point to '+cities[4])
maps[cities[4]]

Distance from Central point to Delhi


#### *The Central Hub point for Delhi seems bit far location for some mobile shops. So, We can create a cluster of 3 in the above map. We will find a central point for each cluster to determine the expo location for delhi*

#### **K-Means Clustering Algorithm**

*This Machine Learning algorithm is used to find the cluster of mobile shops. By obervation from the above map, we can initialize k with 3 clusters.*

In [171]:
from sklearn.cluster import KMeans  # importing Kmeans algorithm from sklearn

In [172]:
#Creating a new dataframe only for delhi data .
# df5 contains data for delhi city. Which we have used earlier during the preprocessing of data.
dfn = df5[['Lat','Lng','Name']]  #our new dataframe consisting of latitude, longitude and name values
dfn.head()

,Lat,Lng,Name
0,28.683666,77.196670,My Idea Store
1,28.535816,77.196995,My Idea Store
2,28.713319,77.109163,My Idea Store
3,28.649738,77.304540,My Idea Store
4,28.641635,77.295433,Trendy cover for HTC


In [173]:
# set number of clusters
kclusters = 3

dfn_k = dfn[['Lat','Lng']]
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfn_k)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([0, 1, 0, 1, 1, 0, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 0, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1,
       0, 0, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1,
       1, 1, 0, 1, 1, 2, 1, 2, 2, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2], dtype=int32)

In [174]:
# merging the cluster labels with the original dataframe
dfn.insert(0, 'Cluster Labels', kmeans.labels_)
dfn.head()

,Cluster Labels,Lat,Lng,Name
0,0,28.683666,77.196670,My Idea Store
1,1,28.535816,77.196995,My Idea Store
2,0,28.713319,77.109163,My Idea Store
3,1,28.649738,77.304540,My Idea Store
4,1,28.641635,77.295433,Trendy cover for HTC


In [175]:
# finding the central hub point for each clusters.
dfn['Cluster Labels'].value_counts()
dfn_coords = dfn.groupby('Cluster Labels').mean()
dfn_coords

,Lat,Lng
Cluster Labels,,
0,28.693307,77.147020
1,28.593265,77.242898
2,28.619456,77.064368


#### **Visualization of Clustered Points with the Central hub point**

In [177]:
# Final Visualization of the cluster with its central point
city = 'Delhi'
latitude = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                    results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
longitude = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                    results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
# create map
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = ['red','green','blue','yellow']
for lat, lon, poi, cluster in zip(dfn['Lat'], dfn['Lng'], dfn['Name'], dfn['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=markers_colors[cluster],
        fill=True,
        fill_color=markers_colors[cluster],
        fill_opacity=0.5).add_to(map_delhi)
    
#add central hubs to each clusters 
for i,j in enumerate(dfn_coords.values):
    folium.CircleMarker( 
            [j[0], j[1]], 
            radius=7,
            color=markers_colors[i],
            fill=True,
            fill_color=markers_colors[3],
            fill_opacity=0.7,
            parse_html=False).add_to(map_delhi)
map_delhi

#### **We can see that the 3 clusters has its own central hub point, which is highlighted using the yellow color. Each cluster points are nearby to the central hub point. So, the Expo can be started in any of the 3 hub points.**

*Thank You*